<h2> Reference </h2>
* [Read images into python environment using OpenCV](https://docs.opencv.org/3.4.0/da/d6e/tutorial_py_geometric_transformations.html)
*  [Create a virtual environment for running python in local](https://stackoverflow.com/questions/32167418/python-pip-install-trouble-shooting-permissionerror-winerror-5-access-is)
    

In [ ]:
import tensorflow as tf
from keras.applications import xception
from keras.preprocessing import image
import keras.preprocessing.image
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import sklearn.ensemble
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
from glob import glob
import cv2
import os
import seaborn as sns
import mpl_toolkits.axes_grid1
import matplotlib
import matplotlib.pyplot as plt
import datetime

%matplotlib inline 
plt.rcParams['figure.figsize'] = [16,10]
plt.rcParams['font.size'] = 16

#start timer
global_start = datetime.datetime.now();

#validation set size
valid_set_size_percentage = 10; # default = 10%

#kaggle
running_on_kaggle = True # set True when running on kaggle environment

#train data
take_only_samples_of_train_data = True; #set False to train on all train data
num_samples_of_train_data_per_species = 200 # < 221, ignored if take_only_samples_of_train_data = True
load_bf_of_train_data = False # set True to load bottleneck features from file

#test data
take_only_samples_of_test_data = False; # set False to predict on all test data
num_samples_of_test_data = 200 # < 794, ignored if take_only_samples_of_test_data = True
load_bf_of_test_data = False # set True to load bottleneck features from file

#augmented imagess
use_aug_data = False # set False to reduce notebook runtime
load_bf_of_aug_data = True # set True to load bottleneck features from file

#show plots
show_plots = True # set False to reduct notebook running time

#overview of directories
print('current directory : ')
!ls
print('\nparent directory : ')
!ls ..

<h2>2.  Analyze Data</h2>

* 12 different plan species
* at least 211 images per species ==> 4750 images in training set
* image sizes vary stringly ==> resize images to (299,299,3) for Xception input

In [ ]:
# input directory
print('input directory : ')
!ls ../input
print('\nfolders containing images of corresponding species')
!ls ../input/train/

In [ ]:
# read train and test data

# directiories
cw_dir = os.getcwd()
data_dir = '../input/'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

# different species in the data set
species = ['Black-grass',
          'Common Chickweed',
          'Loose Silky-bent',
          'Shepherds Purse',
          'Charlock',
          'Common wheat',
          'Maize',
          'Small-flowered Cranesbill',
          'Cleavers',
          'Fat Hen',
          'Scentless Mayweed',
          'Sugar beet']
num_species = len(species)

# print number of images of each species in the training data
for sp in species:
    print('{} images of {}'.format(len(os.listdir(os.path.join(train_dir,sp))), sp))

# read all train data
train = []
for species_id, sp in enumerate(species):
    #print(species_id, sp)
    for file in os.listdir(os.path.join(train_dir, sp)):
        train.append(['train/{}/{}'.format(sp, file), file, species_id, sp])
train_df = pd.DataFrame(train, columns=['filepath',
                                       'file',
                                       'species_id',
                                       'species'])
print('\ntrain_df.shape = ', train_df.shape)

# read all test data
test = []
for file in os.listdir(test_dir):
    test.append(['test/{}'.format(file), file])
test_df  = pd.DataFrame(test, columns = ['filepath','file'])
print('test_df.shape = ', test_df.shape)

# function to read an image 
def read_img(filepath, target_size=None):
    img = cv2.imread(os.path.join(data_dir, filepath), cv2.IMREAD_COLOR)
    img = cv2.resize(img.copy(), target_size, interpolation = cv2.INTER_AREA)
    #img = image.load_img(os.path.join(data_dir, filepath),target_size=target_size)
    #img = image.img_to_array(img)
    return img

In [ ]:
## get all image shapes: this is time-consuming, therefore I have deactivated
## the code. The results are included below and show that all
## images are quadratic, but their size can be very different. 

if True:
    train_df['image_heigth'] = 0
    train_df['image_width'] = 0
    train_df['image_channel'] = 0

    #get all image shapes
    for i in range(len(train_df)):
        img = read_img(train_df.filepath.values[i])
        train_df.loc[i,'image_heigth'] = img.shape[0]
        train_df.loc[i,'image_width'] = img.shape[1]
        train_df.loc[i,'image_channel'] = img.shape[2]

    test_df['image_heigth'] = 0
    test_df['image_width'] = 0
    test_df['image_channel'] = 0

    # get all image shapes
    for i in range(len(test_df)):
        img = read_img(test_df.filepath.values[i])
        test_df.loc[i,'image_heigth'] = img.shape[0]
        test_df.loc[i,'image_width'] = img.shape[1]
        test_df.loc[i,'image_channel'] = img.shape[2]

<h3>Debug error with OpenCV</h3>

[/io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215) dsize.area() > 0 || (inv_scale_x > 0 && inv_scale_y > 0) in function resize
](https://stackoverflow.com/questions/31996367/opencv-resize-fails-on-large-image-with-error-215-ssize-area-0-in-funct)